In [25]:
from pyomo.environ import *
import numpy as np

class OptiModel:
    
    def __init__(self):
        self.m = ConcreteModel()
        self.m.dual = Suffix(direction=Suffix.IMPORT)
        self.m.theta1 = Var(initialize=0.0, domain=NonNegativeReals)
        self.m.theta2 = Var(initialize=0.0, domain=NonNegativeReals)
        self.m.theta1.fixed = True
        self.m.theta2.fixed = True
        self.m.x1 = Var(domain=NonNegativeReals)
        self.m.x2 = Var(domain=NonNegativeReals)
        solverpath = 'C:\\w64\\glpsol'
        self.solver = SolverFactory('glpk', executable=solverpath)
        
        self.m.constraints = ConstraintList()
        constraint_list = [
            self.c1_lhs(),
            self.c2_lhs(),
            self.c3_lhs()
        ]
        for c in constraint_list:
            c_tuple = self.to_tuple(c)
            self.m.constraints.add(c_tuple)
            
        self.m.obj = Objective(expr=self.obj_expr())

        # !!! important to know if c.expr gives all constraints for indexed constraints
        # the answer is no! See notion link.
        
    def c1_lhs(self):
        return 0.8*self.m.x1 + 0.44*self.m.x2 - 24000.0 - self.m.theta1
        
    def c2_lhs(self):
        return 0.05*self.m.x1 + 0.1*self.m.x2 - 2000.0 - self.m.theta2
        
    def c3_lhs(self):
        return 0.1*self.m.x1 + 0.36*self.m.x2 - 6000.0

    def obj_expr(self):
        return - 8.1*self.m.x1 - 10.8*self.m.x2
    
    def to_tuple(self, lhs):
        return (None, lhs, 0.0)
        
    def solve(self):
        self.solver.solve(self.m, tee=True)
    
    def write(self):
        print(self.m.x1.value, self.m.x2.value)

In [26]:
mymodel = OptiModel()
mymodel.solve()
mymodel.write()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\user1\AppData\Local\Temp\tmp9bqq8r_b.glpk.raw --wglp C:\Users\user1\AppData\Local\Temp\tmpq3oq649x.glpk.glp
 --cpxlp C:\Users\user1\AppData\Local\Temp\tmptcrmr5ux.pyomo.lp
Reading problem data from 'C:\Users\user1\AppData\Local\Temp\tmptcrmr5ux.pyomo.lp'...
4 rows, 3 columns, 7 non-zeros
31 lines were read
Writing problem data to 'C:\Users\user1\AppData\Local\Temp\tmpq3oq649x.glpk.glp'...
23 lines were written
GLPK Simplex Optimizer, v4.65
4 rows, 3 columns, 7 non-zeros
Preprocessing...
3 rows, 2 columns, 6 non-zeros
Scaling...
 A: min|aij| =  5.000e-02  max|aij| =  8.000e-01  ratio =  1.600e+01
GM: min|aij| =  6.252e-01  max|aij| =  1.600e+00  ratio =  2.558e+00
EQ: min|aij| =  3.909e-01  max|aij| =  1.000e+00  ratio =  2.558e+00
Constructing initial basis...
Size of triangular part is 3
*     0: obj =   0.000000000e+00 inf =   0.000e+00 (2)
*     2: obj =  -2.867586207e+05 inf =   0.000e+0

decompose term:

In [27]:
expr.current.decompose_term(mymodel.m.constraints[1]._body)

(True,
 [(0.8, <pyomo.core.base.var.SimpleVar at 0x1a2c24540b8>),
  (0.44, <pyomo.core.base.var.SimpleVar at 0x1a2c2454128>),
  (-24000.0, None),
  (-1, <pyomo.core.base.var.SimpleVar at 0x1a2c24339e8>)])